In [1]:
import requests
from bs4 import BeautifulSoup

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from scipy.spatial import distance_matrix

import os
import re
import string
import csv
import numpy as np
import csv 

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def web_crawler(url):
    # Send a GET request to the URL
    session = requests.Session()
    response = session.get(url, headers={'User-Agent': 'Mozilla/5.0'})

    # Parse the HTML content of the response using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

In [3]:
# Main URL
url = 'https://en.wikipedia.org/wiki/Lists_of_diseases'

soup = web_crawler(url)
section1_links = soup.find('div', {'class':'mw-content-ltr mw-parser-output'}).find_all('a', href=True)[32:49]
origin = 'https://en.wikipedia.org/'

urls = [origin + section1_links[i]['href'] for i in range(len(section1_links))]

In [5]:
len(urls)

17

In [88]:
def extract_symptom(url_disease_links):
    dict = {}
    has_symptom = None

    for url_disease_link in url_disease_links:
        symptoms = list()
        soup = web_crawler(url_disease_link)
        has_symptom = soup.find('span', { 'class':'mw-headline', 'id': 'Signs_and_symptoms'})
        if has_symptom:
            for element in has_symptom.parent.next_siblings:
                if element.name == 'p':
                    symptoms.append(element.get_text())
                if element.name == 'h2':
                    break
            disease = url_disease_link.split('/')[-1]
            dict[f'{disease}'] = symptoms
    return dict

In [188]:
def write_file(url_i_disease_links, i):
    fname = 'url_i_disease_links'.replace('i', f'{i}', 1)
    with open(f'{fname}.csv', 'w', encoding='utf-8', newline='') as f:
        wrtr = csv.writer(f)
        wrtr.writerow(['disease', 'symptoms'])
        dict = extract_symptom(url_i_disease_links)
        for k,v in dict.items():
            wrtr.writerow([k,v])

In [184]:
url_0_disease_links = list()
soup = web_crawler(urls[0])
for i in range (1,132):
    t = soup.find('div', {'class': "mw-content-container"}).find_all('tr')[i].find('a', href=True)
    if t != None:
        url_0_disease_links.append(origin + t['href'])

write_file(url_0_disease_links, 0)

In [111]:
url_1_disease_links = list()
soup = web_crawler(urls[1])

for i in soup.find_all('h2'):
    next_ul = i.find_next_sibling('ul')
    if next_ul:
        links = next_ul.find_all('a', href=True)
        for a in links:
            if a and a['href']:
                url_1_disease_links.append(origin + a['href'])
   
write_file(url_1_disease_links, 1)

In [190]:
url_2_disease_links = list()
soup = web_crawler(urls[2])

list_siblings = soup.find_all('h2')[1].find_next_siblings()
for link in list_siblings[4].find_all('a') + list_siblings[7].find_all('a'):
    url_2_disease_links.append(origin + link['href'])

write_file(url_2_disease_links, 2)

In [200]:
url_3_disease_links = list()
soup = web_crawler(urls[3])

diseases_section = soup.find(id='List_of_diseases')
if diseases_section:
        list_items = diseases_section.find_all_next('li')
        for li in list_items:
            a_tags = li.find_all('a')
            for a in a_tags:
                if 'href' in a.attrs:
                    url_3_disease_links.append(origin + a['href']) 

for link in url_3_disease_links[:150]:
    if len(link.split('#')) >1 :
        url_3_disease_links.remove(link)

write_file(url_3_disease_links, 3)        


In [226]:
url_5_disease_links = list()
soup = web_crawler(urls[5])

starting_point = soup.find('span', id='H00-H06_Disorders_of_eyelid,_lacrimal_system_and_orbit')
if starting_point:
    # Find the next unordered list (ul) that contains the list items (li)
    current_point = starting_point.find_next('ul')
    while current_point and current_point.name == 'ul':
        for li in current_point.find_all('li', recursive=True):  # Change to True to get nested lists
            a_tag = li.find('a')
            if a_tag and 'href' in a_tag.attrs:
                url_5_disease_links.append(origin + a_tag['href'])
        
        current_point = current_point.find_next_sibling()

        # Check if we have reached the section containing "Argyll Robertson pupil"
        if any("Argyll Robertson pupil" in s for s in current_point.stripped_strings):
            # Capture the link for "Argyll Robertson pupil"
            a_tag = current_point.find('a', string="Argyll Robertson pupil")
            if a_tag and 'href' in a_tag.attrs:
                full_url = f"https://en.wikipedia.org{a_tag['href']}"
                url_5_disease_links.append(full_url)
            break


In [227]:
start_section_id = 'H00-H06_Disorders_of_eyelid,_lacrimal_system_and_orbit'
end_section_id = 'H55-H59_Other_disorders_of_eye_and_adnexa'

starting_point = soup.find(id=start_section_id)

if starting_point:
    # Find the next unordered list (ul) that contains the list items (li)
    current_point = starting_point.find_next('ul')
    while current_point and current_point.name == 'ul':
        for li in current_point.find_all('li', recursive=True):
            a_tag = li.find('a')
            url_5_disease_links.append(origin + a_tag['href'])

        current_point = current_point.find_next_sibling()

        # Check if we have reached the end section id
        if current_point and current_point.find(id=end_section_id):
            break

In [233]:
start_section_id = 'H00-H06_Disorders_of_eyelid,_lacrimal_system_and_orbit'
end_section_id = 'H30-H36_Disorders_of_choroid_and_retina'

start_section = soup.find(id=start_section_id)
end_section = soup.find(id=end_section_id)

if start_section and end_section:
    content = start_section.find_next('ul').find_all('li', recursive=True)  # Changed to include sub-items

    for li in content:
        a_tag = li.find('a')
        if a_tag and 'href' in a_tag.attrs:
            full_url = f"https://en.wikipedia.org{a_tag['href']}"
            url_5_disease_links.append(full_url)

        if li.find_next_sibling(['h2', 'h3']) is not None:
            next_header = li.find_next_sibling(['h2', 'h3'])
            if next_header and end_section_id in next_header.get('id', ''):
                break

print(len(url_5_disease_links))

write_file(url_5_disease_links, 5)


78


In [254]:
url_8_disease_links = list()
soup = web_crawler(urls[8])
for i in range(71):
    t = soup.find('div', {'class': "mw-content-ltr mw-parser-output"}).find_all('tr')[i].find('a', href=True)
    if t != None:
        url_8_disease_links.append(origin + t['href'])

write_file(url_8_disease_links, 8)

In [264]:
url_16_disease_links = list()
soup = web_crawler(urls[16])

for i in range(243):
    t = soup.find('div', {'class': "mw-content-ltr mw-parser-output"}).find_all('tr')[i].find('a', href=True)
    if t != None:
        url_16_disease_links.append(origin + t['href'])

write_file(url_16_disease_links, 16)

In [267]:
write_file(url_16_disease_links, 16)